In [ ]:

import os
HOME = os.getcwd()
print(HOME)


In [ ]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image


In [ ]:
%cd {HOME}
# !yolo task=detect mode=predict model=yolo12s.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True
!yolo task=detect mode=predict model=yolo11n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

In [ ]:

!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet
# from roboflow import Roboflow
# rf = Roboflow(api_key="n1uxORimJorBfZhVg6ym")
# project = rf.workspace("ravindu-86rcc").project("aisee-test-3jcz4")
# version = project.version(3)
# dataset = version.download("yolov12")

# from roboflow import Roboflow
# rf = Roboflow(api_key="n1uxORimJorBfZhVg6ym")
# project = rf.workspace("ravindu-86rcc").project("aisee-test-3jcz4")
# version = project.version(3)
# dataset = version.download("yolov11")

# from roboflow import Roboflow
# rf = Roboflow(api_key="n1uxORimJorBfZhVg6ym")
# project = rf.workspace("ravindu-86rcc").project("aisee-test-3jcz4")
# version = project.version(6)
# dataset = version.download("yolov11")

from roboflow import Roboflow
rf = Roboflow(api_key="n1uxORimJorBfZhVg6ym")
project = rf.workspace("ravindu-86rcc").project("aisee-test-3jcz4")
version = project.version(7)
dataset = version.download("yolov11")


In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolo11n.pt data=/content/datasets/aisee-test-7/data.yaml epochs=100 imgsz=640 plots=True

In [ ]:
from IPython.display import FileLink, HTML
p = "/content/runs/detect/train/weights/best.pt"
FileLink(p)  # clickable link in the output


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/runs/detect/train/weights/best2.pt" "/content/drive/MyDrive/best2.pt"